Fit a logistic regression model to classify neural events using the fourier data from eeg signals

In [1]:
import numpy as np
import pandas as pd
import sklearn.datasets as skdata
import sklearn.metrics as skmetrics
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression

In [5]:
# MAKE SURE TO CHANGE THIS TO THE LOCAL PATH TO DATA
#TRAIN_PATH = "../augmented_train/reformatted_train_10000_samples.csv"
TRAIN_PATH = "train_10000_samples_0_to_10_hz.csv"

data = pd.read_csv(TRAIN_PATH)

data_np = data.to_numpy()

print(data_np)
print(data_np.shape)

[[2165.875244140625 49192.8300358057 4950.520985686926 ...
  1.0998884090695715 -0.3377874641546745 'LRDA']
 [466.188720703125 34865.269945025444 5704.912146116356 ...
  -0.6758465848267224 -0.4742115666873585 'LPD']
 [196.77908325195312 18550.839977597818 3356.709263537125 ...
  0.7214381887485093 0.5082022822209593 'GPD']
 ...
 [14016254.0 1344455.0649087245 429328.468060573 ... -2.8103673961549203
  -2.122763833527274 'Seizure']
 [726.7728271484375 68503.0798098743 6331.501753051206 ...
  1.2531196335280073 -2.844648354805996 'Seizure']
 [484.8861083984375 48225.09995627403 8512.202587755772 ...
  -0.9684973867015032 -1.878838148097688 'Other']]
(9940, 2421)


In [6]:
num_input = data_np.shape[1] - 1

x = data_np[:, 0:num_input]
y = data_np[:, num_input]

# Shuffle the dataset based on sample indices
shuffled_indices = np.random.permutation(x.shape[0])

# Choose the first 80% as training set, next 10% as validation and the rest as testing
train_split_idx = int(0.80 * x.shape[0])
val_split_idx = int(0.90 * x.shape[0])

train_indices = shuffled_indices[0:train_split_idx]
val_indices = shuffled_indices[train_split_idx:val_split_idx]
test_indices = shuffled_indices[val_split_idx:]

# Select the examples from x and y to construct our training, validation, testing sets
x_train, y_train = x[train_indices, :], y[train_indices]
x_val, y_val = x[val_indices, :], y[val_indices]
x_test, y_test = x[test_indices, :], y[test_indices]

IndexError: index 2421 is out of bounds for axis 1 with size 2421

In [ ]:
# Create the model
model_scikit = LogisticRegression()

# Train
model_scikit.fit(x_train, y_train)
predictions_train = model_scikit.predict(x_train)
score_train = model_scikit.score(x_train, y_train)
print('Training Accuracy: {:.4f}'.format(score_train))

# Validate
predictions_val = model_scikit.predict(x_val)
score_val = model_scikit.score(x_val, y_val)
print('Validation Accuracy: {:.4f}'.format(score_val))

# Test
predictions_test = model_scikit.predict(x_test)
score_test = model_scikit.score(x_test, y_test)
print('Test Accuracy: {:.4f}'.format(score_test))

ValueError: Unknown label type: 'unknown'

In [ ]:
# Test out a number of models
models = []
val_scores = []
solvers = ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga']  
num_iterations = [100, 150, 200, 500]

for solver in solvers:

    for iter in num_iterations:

        model = LogisticRegression(solver=solver, max_iter = iter)

        model.fit(x_train, y_train)

        models.append(model)

        predictions_train = model.predict(x_train)
        score_train = model.score(x_train, y_train)

        predictions_val = model.predict(x_val)
        score_val = model.score(x_val, y_val)

        val_scores.append(score_val)

        print('Solver: {}, Iterations: {}'.format(solver, iter))
        print('Training accuracy: {:0.5f}  Validation accuracy: {:0.2f}%'.format(score_train*100, score_val*100))

# Choose the best model based on highest validation accuracy
print(val_scores)
best_model_idx = np.argmax(val_scores)
best_model = models[best_model_idx]


# Sanity check
print('\nBest solver: {}'.format(solvers[best_model_idx // len(num_iterations)]))
print('Best num iterations: {}'.format(num_iterations[best_model_idx % len(num_iterations)]))

# Best solver on the test set
predictions_test = best_model.predict(x_test)
score_test = best_model.score(x_test, y_test)

print("The test results.... {:0.2f}%".format(score_test*100))

c:\Users\brenn\anaconda3\envs\ml_new\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Solver: liblinear, Iterations: 100
Training accuracy: 37.50000  Validation accuracy: 29.00%


c:\Users\brenn\anaconda3\envs\ml_new\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Solver: liblinear, Iterations: 150
Training accuracy: 42.87500  Validation accuracy: 28.00%


c:\Users\brenn\anaconda3\envs\ml_new\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


...

c:\Users\brenn\anaconda3\envs\ml_new\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Solver: saga, Iterations: 150
Training accuracy: 17.62500  Validation accuracy: 22.00%


c:\Users\brenn\anaconda3\envs\ml_new\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Solver: saga, Iterations: 200
Training accuracy: 18.12500  Validation accuracy: 25.00%
Solver: saga, Iterations: 500
Training accuracy: 18.50000  Validation accuracy: 23.00%
[0.29, 0.28, 0.32, 0.27, 0.32, 0.28, 0.3, 0.32, 0.24, 0.23, 0.23, 0.31, 0.25, 0.24, 0.23, 0.23, 0.25, 0.22, 0.25, 0.23]

Best solver: liblinear
Best num iterations: 200
The test results.... 32.00%


c:\Users\brenn\anaconda3\envs\ml_new\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [ ]:
cm = skmetrics.confusion_matrix(y_test, predictions_test)
plt.figure(figsize=(9,9))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r')
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
all_sample_title = 'Accuracy Score: {0}'.format(score_test)
plt.title(all_sample_title, size = 15)

NameError: name 'skmetrics' is not defined

In [ ]:
kullback_leibler_train = skmetrics.mutual_info_score(y_train, predictions_train)
kullback_leibler_test = skmetrics.mutual_info_score(y_test, predictions_test)

print(kullback_leibler_train)
print(kullback_leibler_test)

0.00435628926865915
0.22469821037301801


In [ ]:
TRAIN_PATH = "reformatted_train_10000_samples.csv"

data = pd.read_csv(TRAIN_PATH)

data_np = data.to_numpy()

print(data_np)
print(data_np.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'reformatted_train_10000_samples.csv'

In [ ]:
x = data_np[:, 0:820]
y = data_np[:, 820]

# Shuffle the dataset based on sample indices
shuffled_indices = np.random.permutation(x.shape[0])

# Choose the first 80% as training set, next 10% as validation and the rest as testing
train_split_idx = int(0.80 * x.shape[0])
val_split_idx = int(0.90 * x.shape[0])

train_indices = shuffled_indices[0:train_split_idx]
val_indices = shuffled_indices[train_split_idx:val_split_idx]
test_indices = shuffled_indices[val_split_idx:]

# Select the examples from x and y to construct our training, validation, testing sets
x_train, y_train = x[train_indices, :], y[train_indices]
x_val, y_val = x[val_indices, :], y[val_indices]
x_test, y_test = x[test_indices, :], y[test_indices]

In [ ]:
# Rerun on the optimal dataset
max_iter = 150
solver = 'newton-cg'

model = LogisticRegression(solver=solver, max_iter=max_iter) 

model.fit(x_train, y_train)

predictions_train = model.predict(x_train)
score_train = model.score(x_train, y_train)

predictions_val = model.predict(x_val)
score_val = model.score(x_val, y_val)

predictions_test = model.predict(x_test)
score_test = model.score(x_test, y_test)

print('Solver: {}, Iterations: {}'.format(solver, max_iter))
print('Training accuracy: {:0.2f}%\nValidation accuracy: {:0.2f}%\nTest accuracy: {:0.2f}%'.format(score_train*100, score_val*100, score_test*100))

c:\Users\brenn\anaconda3\envs\ml_new\lib\site-packages\sklearn\utils\optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(


Solver: newton-cg, Iterations: 150
Training accuracy: 59.46%
Validation accuracy: 54.38%\Test accuracy: 54.33%
